In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
import seaborn as sns

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from bespoke.blaze_preapprovals import blaze_uw as blaze_uw

%load_ext autoreload
%autoreload 2

# ~ Part 1: Create UW data ~
This part is to read multiple queries to pull all the relevant data needed to perform underwriting.

## Gross Margin

In [3]:
blaze_gm_data = pd.read_sql_query(blaze_uw.create_blaze_gm_query('MA'),engine)
blaze_gm_data['year_month'] = pd.to_datetime(blaze_gm_data['year_month'])

In [4]:
# Check N of shops to underwrite
blaze_gm_data[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,6
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,10
2022-03-31,8
2022-04-30,10


In [5]:
# only underwrite those with more than 5 months of history data
blaze_gm_data_long = blaze_gm_data[blaze_gm_data['month_available'] >= 5]
blaze_gm_data_long[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,5
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,9
2022-03-31,8
2022-04-30,10


In [7]:
gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])

## Monthly GMV change

## state website published benchmark

In [11]:
rev_change_bm_with_weight['year_month'] = gm_distr_all['year_month'].values

In [12]:
rev_change_bm_with_weight

,0,year_month
0,NaN,2021-08-31
1,-0.07,2021-09-30
2,0.01,2021-10-31
3,-0.07,2021-11-30
4,0.09,2021-12-31
5,0.07,2022-01-31
6,0.02,2022-02-28
7,0.11,2022-03-31
8,-0.01,2022-04-30
9,-0.05,2022-05-31


In [13]:
# here for MA we use our own bm, so this part has to be changed manually ...

rev_change_bm_with_weight = pd.DataFrame([numpy.nan,-0.07, 0.01, -0.07, 0.09,0.07,0.02,0.11,-0.01,-0.05,-0.01,0.07,-0.02,-0.07,-0.04,-0.04,numpy.nan])
rev_change_bm_with_weight['year_month'] = gm_distr_all['year_month'].values
rev_change_bm_with_weight['weight'] = [numpy.nan,numpy.nan,numpy.nan,0.04,0.08,0.12,0.2,0.28,0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
rev_change_bm_with_weight.columns = ['rev_change_bm','year_month','rev_change_weight']
rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
rev_change_bm_with_weight

,rev_change_bm,year_month,rev_change_weight
0,NaN,2021-08-31,NaN
1,-0.07,2021-09-30,NaN
2,0.01,2021-10-31,NaN
3,-0.07,2021-11-30,0.04
4,0.09,2021-12-31,0.08
5,0.07,2022-01-31,0.12
6,0.02,2022-02-28,0.20
7,0.11,2022-03-31,0.28
8,-0.01,2022-04-30,0.36
9,-0.05,2022-05-31,0.44


In [14]:

blaze_gm_data_long_with_rev  = pd.merge(blaze_gm_data_long,rev_change_bm_with_weight,on=['year_month'], how='inner')
blaze_gm_data_long_with_rev['rev_change_var'] = blaze_gm_data_long_with_rev['rev_change'] - blaze_gm_data_long_with_rev['rev_change_bm']
blaze_gm_data_long_with_rev['rev_change_points'] = [blaze_uw.gmv_change_variance_point_mapping(n) for n in blaze_gm_data_long_with_rev['rev_change_var']]
blaze_gm_data_long_with_rev['rev_change_total'] = blaze_gm_data_long_with_rev['rev_change_points'] * blaze_gm_data_long_with_rev['rev_change_weight']



In [15]:
blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['year_month']=='2022-12-31']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
135,2022-12-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,409537.45,313229.31,198890.48,210646.97,0.514353,17,503997.87,0.535636,0.535954,0.520750,0.307468,NaN,1.0,NaN,NaN,NaN
136,2022-12-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,378817.29,280400.53,180526.08,198291.21,0.523448,17,466541.61,0.525254,0.512104,0.492689,0.350986,NaN,1.0,NaN,NaN,NaN
137,2022-12-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,468802.61,427813.25,191382.49,277420.12,0.591763,17,584051.52,0.600879,0.592161,0.558665,0.095811,NaN,1.0,NaN,NaN,NaN
138,2022-12-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,828146.02,688169.09,390795.93,437350.09,0.528107,17,1124238.73,0.533876,0.528034,0.505312,0.203405,NaN,1.0,NaN,NaN,NaN
139,2022-12-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,170217.70,136908.95,75149.80,95067.90,0.558508,10,214030.37,0.538263,0.527626,0.437876,0.243291,NaN,1.0,NaN,NaN,NaN
140,2022-12-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,248281.57,192265.80,116427.05,131854.52,0.531068,9,305787.05,0.542874,0.540356,0.496514,0.291345,NaN,1.0,NaN,NaN,NaN
141,2022-12-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,433620.64,352741.27,195939.44,237681.20,0.548132,13,544331.81,0.541151,0.531741,0.527765,0.229288,NaN,1.0,NaN,NaN,NaN
142,2022-12-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,459015.89,364645.93,278416.61,180599.28,0.393449,13,714057.68,0.402100,0.419868,0.419805,0.258799,NaN,1.0,NaN,NaN,NaN
143,2022-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,580418.59,456312.77,305661.44,274757.15,0.473378,15,760524.72,0.508980,0.504500,0.494312,0.271975,NaN,1.0,NaN,NaN,NaN
144,2022-12-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,546591.47,424641.90,284332.44,262259.03,0.479808,17,730950.52,0.503389,0.500059,0.495201,0.287182,NaN,1.0,NaN,NaN,NaN


In [16]:
blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['shopid']== '623e39dfbcd2bd3c1fb82f46']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
60,2022-04-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,53.00,NaN,43.00,10.00,0.188679,9,43.00,0.188679,0.188679,0.188679,NaN,-0.01,0.36,NaN,NaN,NaN
70,2022-05-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,39550.45,53.00,19064.33,20486.12,0.517974,9,19107.33,0.353327,0.353327,0.353327,745.234906,-0.05,0.44,745.284906,NaN,NaN
80,2022-06-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,108733.99,39550.45,52210.39,56523.60,0.519834,9,71317.72,0.408829,0.408829,0.408829,1.749248,-0.01,0.52,1.759248,NaN,NaN
90,2022-07-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,152905.36,108733.99,72793.09,80112.27,0.523934,9,144067.81,0.520581,0.437605,0.437605,0.406233,0.07,0.60,0.336233,10.0,6.0
100,2022-08-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,151454.24,152905.36,69419.55,82034.69,0.541647,9,194423.03,0.528471,0.458414,0.458414,-0.009490,-0.02,0.68,0.010510,5.0,3.4
110,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,188028.92,151454.24,85000.93,103027.99,0.547937,9,227213.57,0.537839,0.473334,0.473334,0.241490,-0.07,0.76,0.311490,10.0,7.6
120,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,227504.01,188028.92,102331.79,125172.22,0.550198,9,256752.27,0.546594,0.533587,0.484315,0.209942,-0.04,0.84,0.249942,10.0,8.4
130,2022-11-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,192265.80,227504.01,87028.21,105237.59,0.547355,9,274360.93,0.548497,0.538484,0.492195,-0.154891,-0.04,0.92,-0.114891,-5.0,-4.6
140,2022-12-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,248281.57,192265.80,116427.05,131854.52,0.531068,9,305787.05,0.542874,0.540356,0.496514,0.291345,NaN,1.00,NaN,NaN,NaN


## Inventory

In [17]:
#need to define the uw date first
uw_date = '2022-12-31'

In [18]:
inventory_date_list = pd.read_sql_query(blaze_uw.create_blaze_inventory_date_query('MA',uw_date),engine)

In [19]:
blaze_inv_data_all = pd.read_sql_query(blaze_uw.create_blaze_inventory_query('MA',tuple([str(x) for x in inventory_date_list['date']]),uw_date),engine)
blaze_inv_data_all['year_month'] = pd.to_datetime(blaze_inv_data_all['year_month'])

In [20]:
#calculate fresh, stale, fresh + stale inventory
fresh_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'fresh'][['year_month','shopid','companyid','inventory']]
stale_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'stale'][['year_month','shopid','companyid','inventory']]
fresh_and_stale_inv = fresh_inv.merge(stale_inv, on = ['shopid','companyid','year_month'],how = 'outer')
fresh_and_stale_inv = fresh_and_stale_inv.fillna(0)
fresh_and_stale_inv.columns = ['year_month','shopid','companyid','inventory_fresh','inventory_stale']
fresh_and_stale_inv['inventory_fresh_and_stale'] = fresh_and_stale_inv['inventory_fresh']  + fresh_and_stale_inv['inventory_stale']

In [21]:
set(blaze_gm_data['shopid']) - set(fresh_and_stale_inv['shopid'])

{'605cef20813d4c08ebf50c06'}

In [22]:
#make sure we do not lack inventory data for too many locations (usually 1 or 2 are fine)
set(blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['year_month'] == uw_date]['shopid']) - set(fresh_and_stale_inv['shopid'])



set()

## Inventory Turnover

In [23]:
data_joined = pd.merge(blaze_gm_data_long_with_rev,fresh_and_stale_inv,on=['year_month','shopid','companyid'], how='inner')



In [24]:
data_joined['inventory_turnover'] = data_joined['sum_cogs_3m'] / data_joined['inventory_fresh_and_stale'] * 4
data_joined['year_month'] = pd.to_datetime(data_joined['year_month'])

In [ ]:
# ##some sanity checks
# data_joined[data_joined['year_month'] >= '2021-10-31'].groupby(['year_month'])['inventory_turnover'].describe().reset_index()
# round(data_joined['inventory_turnover'].quantile(0.95),2)

# ~ Part 2: Start UW ~
Now that we have all the essential data for UW. We choose the date we want to perform underwriting on and start calculating the KPIs.

In [25]:
recent = data_joined[data_joined['year_month'] == '2022-12-31']
recent = recent.reset_index(drop = True)

In [26]:
# check final # of locations to UW
recent[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-12-31,10


## 1. gm % score

In [27]:
gm_threshold = [[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7]]

In [28]:
gm_score_data = recent.apply(lambda row: blaze_uw.get_gm_perc_scores(gm_threshold,row['avg_margin_3m'],row['avg_margin_6m'],row['avg_margin_all']),axis = 1)
recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_3m_score', 'gm_6m_score','gm_all_score'])],axis=1)



## 2. inventory turnover score

In [30]:
recent_with_gm['turnover_score'] = [10 if (recent_with_gm['inventory_turnover'][i] >= 6 and recent_with_gm['inventory_turnover'][i] <= 26) else 0 for i in range(len(recent_with_gm))]



## 3. rev change score

In [31]:
rev_change_score = blaze_gm_data_long_with_rev[['shopid','rev_change_total']].groupby('shopid').sum().reset_index()

In [32]:
rev_change_score['rev_change_score'] = [max(min(round(rev_change_score['rev_change_total'][i],2),10),-10) for i in range(len(rev_change_score))]
#rev_change_score.sort_values(by = 'rev_change_total')


## 4. gm $ score

In [33]:
gm_dollar_score = blaze_gm_data_long_with_rev[['shopid','profit_pre_tax_post_discount']].groupby('shopid').mean().reset_index()

In [34]:
gm_dollar_score['gm_dollar_score'] = [15 if gm_dollar_score['profit_pre_tax_post_discount'][i] >= 200000 else 0 for i in range(len(rev_change_score))]


In [35]:
recent_with_gm_and_rev_change = pd.merge(recent_with_gm,rev_change_score[['shopid','rev_change_score']],on=['shopid'], how='inner')
#recent_with_gm_and_rev_change

# ~ Part 3: Final scoring and credit limit & rate calculation ~

In [36]:
final = pd.merge(recent_with_gm_and_rev_change,gm_dollar_score[['shopid','gm_dollar_score']],on=['shopid'], how='inner')


In [37]:
final['total_gm_perc_score'] = final['gm_3m_score'] + final['gm_6m_score'] + final['gm_all_score']
final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
final['credit_limit_raw'] = [round(min(final['sum_cogs_3m'][i],final['inventory_fresh'][i]),0) for i in range(len(final))]
final['potential_credit_limit'] = round(final['credit_limit_raw'],-4) 
final['initial_credit_limit'] = final['potential_credit_limit'] / 2                         



In [38]:
final

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-12-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,409537.45,313229.31,198890.48,210646.97,0.514353,17,503997.87,0.535636,0.535954,0.520750,0.307468,NaN,1.0,NaN,NaN,NaN,230739.660792,6466.512500,237206.173292,8.498900,5,5,5,10,10,0,15,15,35,230740.0,230000.0,115000.0
1,2022-12-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,378817.29,280400.53,180526.08,198291.21,0.523448,17,466541.61,0.525254,0.512104,0.492689,0.350986,NaN,1.0,NaN,NaN,NaN,311035.133750,13945.370000,324980.503750,5.742395,5,5,0,0,10,0,10,10,20,311035.0,310000.0,155000.0
2,2022-12-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,468802.61,427813.25,191382.49,277420.12,0.591763,17,584051.52,0.600879,0.592161,0.558665,0.095811,NaN,1.0,NaN,NaN,NaN,147678.255000,5029.395000,152707.650000,15.298553,5,5,5,10,10,15,15,15,35,147678.0,150000.0,75000.0
3,2022-12-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,828146.02,688169.09,390795.93,437350.09,0.528107,17,1124238.73,0.533876,0.528034,0.505312,0.203405,NaN,1.0,NaN,NaN,NaN,410290.197500,17334.325000,427624.522500,10.516130,5,5,5,10,10,15,15,15,35,410290.0,410000.0,205000.0
4,2022-12-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,170217.70,136908.95,75149.80,95067.90,0.558508,10,214030.37,0.538263,0.527626,0.437876,0.243291,NaN,1.0,NaN,NaN,NaN,164363.628750,4827.540000,169191.168750,5.060084,5,5,-2,0,10,0,8,8,18,164364.0,160000.0,80000.0
5,2022-12-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,248281.57,192265.80,116427.05,131854.52,0.531068,9,305787.05,0.542874,0.540356,0.496514,0.291345,NaN,1.0,NaN,NaN,NaN,255603.826250,13947.850833,269551.677083,4.537713,5,5,0,0,10,0,10,10,20,255604.0,260000.0,130000.0
6,2022-12-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,433620.64,352741.27,195939.44,237681.20,0.548132,13,544331.81,0.541151,0.531741,0.527765,0.229288,NaN,1.0,NaN,NaN,NaN,217525.426667,11322.890000,228848.316667,9.514281,5,5,5,10,10,0,15,15,35,217525.0,220000.0,110000.0
7,2022-12-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,459015.89,364645.93,278416.61,180599.28,0.393449,13,714057.68,0.402100,0.419868,0.419805,0.258799,NaN,1.0,NaN,NaN,NaN,371571.358345,27126.232500,398697.590845,7.163903,-2,-2,-2,10,10,0,-6,-6,14,371571.0,370000.0,185000.0
8,2022-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,580418.59,456312.77,305661.44,274757.15,0.473378,15,760524.72,0.508980,0.504500,0.494312,0.271975,NaN,1.0,NaN,NaN,NaN,335936.541750,9567.303333,345503.845083,8.804819,5,5,0,10,10,0,10,10,30,335937.0,340000.0,170000.0
9,2022-12-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,546591.47,424641.90,284332.44,262259.03,0.479808,17,730950.52,0.503389,0.500059,0.495201,0.287182,NaN,1.0,NaN,NaN,NaN,223512.647917,2208.586250,225721.234167,12.953155,5,5,0,10,10,15,10,15,35,223513.0,220000.0,110000.0


In [39]:
# look at distribution of total score
final['total_score'].describe()

count    10.000000
mean     27.700000
std       8.641631
min      14.000000
25%      20.000000
50%      32.500000
75%      35.000000
max      35.000000
Name: total_score, dtype: float64

# ~ Part 4: Generate pre-approvals ~
Passing criteria:
1. total score >= 30
2. GM dollar >= 25K
3. Potential credit limit >= 30K

In [40]:
# per passing criterial above
final_pass = final[(final['total_score'] >= 24)&\
                  (final['profit_pre_tax_post_discount'] >= 25000)&\
                  (final['potential_credit_limit'] >= 30000)]
final_pass = final_pass.reset_index(drop=True)

In [41]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-12-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,409537.45,313229.31,198890.48,210646.97,0.514353,17,503997.87,0.535636,0.535954,0.520750,0.307468,NaN,1.0,NaN,NaN,NaN,230739.660792,6466.512500,237206.173292,8.498900,5,5,5,10,10,0,15,15,35,230740.0,230000.0,115000.0
1,2022-12-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,468802.61,427813.25,191382.49,277420.12,0.591763,17,584051.52,0.600879,0.592161,0.558665,0.095811,NaN,1.0,NaN,NaN,NaN,147678.255000,5029.395000,152707.650000,15.298553,5,5,5,10,10,15,15,15,35,147678.0,150000.0,75000.0
2,2022-12-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,828146.02,688169.09,390795.93,437350.09,0.528107,17,1124238.73,0.533876,0.528034,0.505312,0.203405,NaN,1.0,NaN,NaN,NaN,410290.197500,17334.325000,427624.522500,10.516130,5,5,5,10,10,15,15,15,35,410290.0,410000.0,205000.0
3,2022-12-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,433620.64,352741.27,195939.44,237681.20,0.548132,13,544331.81,0.541151,0.531741,0.527765,0.229288,NaN,1.0,NaN,NaN,NaN,217525.426667,11322.890000,228848.316667,9.514281,5,5,5,10,10,0,15,15,35,217525.0,220000.0,110000.0
4,2022-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,580418.59,456312.77,305661.44,274757.15,0.473378,15,760524.72,0.508980,0.504500,0.494312,0.271975,NaN,1.0,NaN,NaN,NaN,335936.541750,9567.303333,345503.845083,8.804819,5,5,0,10,10,0,10,10,30,335937.0,340000.0,170000.0
5,2022-12-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,546591.47,424641.90,284332.44,262259.03,0.479808,17,730950.52,0.503389,0.500059,0.495201,0.287182,NaN,1.0,NaN,NaN,NaN,223512.647917,2208.586250,225721.234167,12.953155,5,5,0,10,10,15,10,15,35,223513.0,220000.0,110000.0


In [42]:
final_pass['annual_rate'] = [blaze_uw.calculate_interest_rate(final_pass['total_score'][i],45)[1] for i in range(len(final_pass))]
final_pass['monthly_rate'] = [blaze_uw.calculate_interest_rate(final_pass['total_score'][i],45)[0] for i in range(len(final_pass))]


In [43]:
# look at the approval rate this month
final_pass.shape[0] / final.shape[0]

0.6

In [44]:
final_pass.shape[0]

6

In [45]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit,annual_rate,monthly_rate
0,2022-12-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,409537.45,313229.31,198890.48,210646.97,0.514353,17,503997.87,0.535636,0.535954,0.520750,0.307468,NaN,1.0,NaN,NaN,NaN,230739.660792,6466.512500,237206.173292,8.498900,5,5,5,10,10,0,15,15,35,230740.0,230000.0,115000.0,0.20,0.0167
1,2022-12-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,468802.61,427813.25,191382.49,277420.12,0.591763,17,584051.52,0.600879,0.592161,0.558665,0.095811,NaN,1.0,NaN,NaN,NaN,147678.255000,5029.395000,152707.650000,15.298553,5,5,5,10,10,15,15,15,35,147678.0,150000.0,75000.0,0.20,0.0167
2,2022-12-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,828146.02,688169.09,390795.93,437350.09,0.528107,17,1124238.73,0.533876,0.528034,0.505312,0.203405,NaN,1.0,NaN,NaN,NaN,410290.197500,17334.325000,427624.522500,10.516130,5,5,5,10,10,15,15,15,35,410290.0,410000.0,205000.0,0.20,0.0167
3,2022-12-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,433620.64,352741.27,195939.44,237681.20,0.548132,13,544331.81,0.541151,0.531741,0.527765,0.229288,NaN,1.0,NaN,NaN,NaN,217525.426667,11322.890000,228848.316667,9.514281,5,5,5,10,10,0,15,15,35,217525.0,220000.0,110000.0,0.20,0.0167
4,2022-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,580418.59,456312.77,305661.44,274757.15,0.473378,15,760524.72,0.508980,0.504500,0.494312,0.271975,NaN,1.0,NaN,NaN,NaN,335936.541750,9567.303333,345503.845083,8.804819,5,5,0,10,10,0,10,10,30,335937.0,340000.0,170000.0,0.21,0.0175
5,2022-12-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,546591.47,424641.90,284332.44,262259.03,0.479808,17,730950.52,0.503389,0.500059,0.495201,0.287182,NaN,1.0,NaN,NaN,NaN,223512.647917,2208.586250,225721.234167,12.953155,5,5,0,10,10,15,10,15,35,223513.0,220000.0,110000.0,0.20,0.0167


# ~ Part 5: output data ~

In [ ]:
# # # only take the essential columns to output to csv file
# final_pass[['shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs',
#        'profit_pre_tax_post_discount', 'margin_pre_tax_post_discount',
#        'sum_cogs_3m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all',
#        'inventory_fresh_and_stale', 'inventory_turnover', 'gm_3m_score',
#        'gm_6m_score', 'gm_all_score', 'total_gm_perc_score', 'gm_dollar_score', 'total_gm_score', 'turnover_score', 'rev_change_score',
# 'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit','annual_rate',
#        'monthly_rate']]

## Output the pre-approval list to a csv file
Always name the csv file as "blaze_ma_MONTH_approval.csv"

In [ ]:
# final_pass[['shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs',
#        'profit_pre_tax_post_discount', 'margin_pre_tax_post_discount',
#        'sum_cogs_3m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all',
#        'inventory_fresh_and_stale', 'inventory_turnover', 'gm_3m_score',
#        'gm_6m_score', 'gm_all_score', 'total_gm_perc_score', 'gm_dollar_score', 'total_gm_score', 'turnover_score', 'rev_change_score',
# 'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit','annual_rate',
#        'monthly_rate']].to_csv('blaze_ma_dec_approval.csv')

## Output all uw data to a csv file
Always name the csv file as "blaze_ma_MONTH_all.csv"

In [ ]:
# final[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
#        'margin_pre_tax_post_discount', 'sum_cogs_3m',
#        'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
#        'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
#        'total_gm_score','turnover_score', 'rev_change_score',  'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit']]\
# .to_csv('blaze_ma_dec_all.csv')


# ~ Part 6: Diff analysis to compare with last month ~

In [46]:
# read last month's files
#prev_approve = pd.read_csv('blaze_ma_oct_approval.csv',index_col=0)
prev_all = pd.read_csv('blaze_ma_nov_all.csv',index_col=0)

In [47]:
prev_all

,year_month,shopid,companyid,rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,total_gm_perc_score,gm_dollar_score,total_gm_score,turnover_score,rev_change_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,437667.52,503458.10,0.534953,1435174.27,0.531575,0.531181,0.519522,0.502901,359163.921000,15.983502,5,5,5,15,15,15,10,10.0,35.0,355654.0,360000.0,180000.0
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,251999.86,265496.52,0.513040,755322.35,0.502620,0.501870,0.504403,0.496247,206025.113000,14.664666,5,5,0,10,15,15,10,10.0,35.0,200066.0,200000.0,100000.0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,161323.42,175887.81,0.521595,512114.76,0.516592,0.517596,0.502646,0.491483,246557.779219,8.308231,5,5,0,10,0,10,10,10.0,30.0,232326.0,230000.0,115000.0
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,168858.98,211206.13,0.555710,502076.65,0.549431,0.548800,0.537614,0.525731,243417.557000,8.250459,5,5,5,15,0,15,10,10.0,35.0,235230.0,240000.0,120000.0
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,239024.96,365484.37,0.604597,791179.75,0.599181,0.594942,0.582123,0.554735,177139.623167,17.865675,5,5,5,15,15,15,10,-1.4,23.6,155182.0,160000.0,80000.0
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,77877.82,88243.34,0.531199,223910.36,0.532557,0.536044,0.518130,0.420323,123983.278000,7.223889,5,5,-2,8,0,8,10,10.0,28.0,117398.0,120000.0,60000.0
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,105636.53,132821.20,0.557001,288509.10,0.555421,0.554925,0.544958,0.494061,184230.271333,6.264098,5,5,0,10,0,10,10,10.0,30.0,180317.0,180000.0,90000.0
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,248635.27,281790.24,0.531253,719671.18,0.515225,0.512478,0.502954,0.495582,263765.440667,10.913806,5,5,0,10,0,10,10,10.0,30.0,251272.0,250000.0,125000.0
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,227042.19,171869.03,0.430845,547276.57,0.461421,0.454801,0.431704,0.432142,287106.939667,7.624707,-2,-2,-2,-6,0,-6,10,10.0,14.0,269630.0,270000.0,135000.0
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,194619.58,226440.81,0.537787,625605.73,0.533178,0.530389,0.524699,0.528190,275401.663000,9.086448,5,5,5,15,0,15,10,10.0,35.0,236329.0,240000.0,120000.0


In [48]:
prev_approve = prev_all[prev_all['total_score'] > 24]

In [49]:
# newly approved this month
newly_added_list = list(set(final_pass['shopid']) - set(prev_approve['shopid']))
final_pass[final_pass['shopid'].isin(newly_added_list)][['shopid','total_score']].sort_values(by = 'shopid')

,shopid,total_score
1,5e7e9d34f795d008e7136fe1,35


In [50]:
# how they scored last month?
prev_all[prev_all['shopid'].isin(newly_added_list)][['shopid','total_score']].sort_values(by = 'shopid')

,shopid,total_score
4,5e7e9d34f795d008e7136fe1,23.6


In [51]:
# dropoffs
dropoff_lsit = list(set(prev_approve['shopid']) - set(final_pass['shopid']))

In [52]:
final[final['shopid'].isin(dropoff_lsit)]

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
1,2022-12-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,378817.29,280400.53,180526.08,198291.21,0.523448,17,466541.61,0.525254,0.512104,0.492689,0.350986,NaN,1.0,NaN,NaN,NaN,311035.13375,13945.370000,324980.503750,5.742395,5,5,0,0,10,0,10,10,20,311035.0,310000.0,155000.0
4,2022-12-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,170217.70,136908.95,75149.80,95067.90,0.558508,10,214030.37,0.538263,0.527626,0.437876,0.243291,NaN,1.0,NaN,NaN,NaN,164363.62875,4827.540000,169191.168750,5.060084,5,5,-2,0,10,0,8,8,18,164364.0,160000.0,80000.0
5,2022-12-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,248281.57,192265.80,116427.05,131854.52,0.531068,9,305787.05,0.542874,0.540356,0.496514,0.291345,NaN,1.0,NaN,NaN,NaN,255603.82625,13947.850833,269551.677083,4.537713,5,5,0,0,10,0,10,10,20,255604.0,260000.0,130000.0


# ~ Part 7: Summary stats ~

In [54]:
print('Total number of shop to UW: ' + str(len(final))+ ' (have at least 6m of data)')
print('Total number of shops approved: ' + str(len(final_pass)))
print('The approval rate is: ' + str(round(len(final_pass)/len(final),2)))
print('Total Final Limit rounded: '+ str(sum(final_pass['potential_credit_limit'])))
print('Average Monthly Rate: ' + str(numpy.mean(final_pass['monthly_rate'])))

Total number of shop to UW: 10 (have at least 6m of data)
Total number of shops approved: 6
The approval rate is: 0.6
Total Final Limit rounded: 1570000.0
Average Monthly Rate: 0.016833333333333336


# ~ scatch, please ignore ~

In [ ]:
# gm_distr_6m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_6m'].describe().reset_index()
# gm_distr_6m.index = pd.to_datetime(gm_distr_6m['year_month'])
# gm_distr_6m
# round(gm_distr_6m[['25%','50%','75%']].mean(),2)
# round(blaze_gm_data_long['avg_margin_6m'].quantile(0.95),2)

In [ ]:
# gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
# gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])
# round(gm_distr_all,3)
# round(gm_distr_all[['25%','50%','75%']].mean(),2)
# round(blaze_gm_data_long['avg_margin_all'].quantile(0.95),2)